In [1]:
import pandas as pd

In [3]:
def extract_metadata(xml_file):
    pass

def read_path(csv_file):
    df = pd.read_csv(csv_file, encoding='utf-8')
    return df


In [4]:
df = read_path('./local_data/table_list_02_dec1.csv')
df.head()

,Image_Id,Path
0,0,unziped_02_dec/tx_7724/29028/images/Tuexen_772...
1,1,unziped_02_dec/tx_7724/29028/images/Tuexen_772...
2,2,unziped_02_dec/tx_7724/29028/images/Tuexen_772...
3,3,unziped_02_dec/tx_7724/29028/images/Tuexen_772...
4,4,unziped_02_dec/tx_7963/29039/images/Tuexen_796...
